In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
ls1 = []
ls2 = []
for dirs in os.listdir('/kaggle/input/pokemon-generation-one/dataset/dataset/'):
    ls1.append(dirs)
    i = 0
    for dir1 in os.listdir('/kaggle/input/pokemon-generation-one/dataset/dataset/' + dirs):
        i += 1
    ls2.append(i)
# Any results you write to the current directory are saved as output.

In [ ]:
print(ls1)
print(ls2)
print(len(ls1))
print(len(ls2))

#如何处理照片数据
#搭建模型
#训练
#测试

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 创建训练集和测试集
os.mkdir('/kaggle/input/train_data')
os.mkdir('/kaggle/input/test_data')
for dirs in os.listdir('/kaggle/input/pokemon-generation-one/dataset/dataset/'):
    print(dirs)

In [ ]:
print(os.path.exists('/kaggle/input/train_data'))
print(os.path.exists('/kaggle/input/test_data'))

In [ ]:
from shutil import copyfile

In [ ]:
dir1 = '/kaggle/input/pokemon-generation-one/dataset/dataset'
dir_train = '/kaggle/input/train_data'
dir_test = '/kaggle/input/test_data'

for dirs in os.listdir(dir1):
    dir2 = dir1 + '/' + dirs
    num = 0
    for dirs1 in os.listdir(dir2):
        num += 1
    print(num)
    
    os.mkdir(dir_train + '/' + dirs)
    os.mkdir(dir_test + '/' + dirs)
    dir_train1 = dir_train + '/' + dirs
    dir_test1 = dir_test + '/' + dirs
    
    i = 0
    
    for dirs2 in os.listdir(dir2):
        dir3 = dir2 + '/' + dirs2
        print(dir3)
        
        if i < 0.6 * num:
            copyfile(dir3, (dir_train1 + '/' + dirs2))
            i += 1
            print(i)
        else:
            copyfile(dir3, (dir_test1 + '/' + dirs2))
            i += 1
    

In [ ]:
for dirs in os.listdir(dir_test):
    print(dirs)

for dirs in os.listdir(dir_train):
    print(dirs)

In [ ]:
# 将多通道的图片变成单通道图片, 并统一像素
def change_picture(dir_data, height, weight):
    for dirs in os.listdir(dir_data):
        dir1 = os.path.join(dir_data, dirs)
        for dir2 in os.listdir(dir1):
            dir3 = os.path.join(dir1, dir2)
            if dir3[-3:] == 'jpg' or dir3[-3] == 'png':
                img = Image.open(dir3)
                gray = img.convert('L')
                gray = gray.resize((height, weight))
                gray.save(dir3)

In [ ]:
change_picture(dir_train, 32, 32)
change_picture(dir_test, 32, 32)

In [ ]:
# name and number dict
name_number_pair = {}
a = 0
for dirs in os.listdir(dir_train):
    if dirs not in name_number_pair:
        name_number_pair[dirs] = a
        a += 1

print(name_number_pair)

In [ ]:
# a dict save the vector representation of the picutres
# {number:[picture1: [height, weight], ...]}

def treat_data(dir_data):
    dic = dict()
    for dirs in os.listdir(dir_data):
        dir1 = os.path.join(dir_data, dirs)
        ls = []
        for dir2 in os.listdir(dir1):
            dir3 = os.path.join(dir1, dir2)
            if dir3[-3:] == 'jpg' or dir3[-3] == 'png':
                img = Image.open(dir3)
                img = np.array(img) / 255 # 归一
                ls.append(img)

        dic[dirs] = ls
    return dic

In [ ]:
representation_train = treat_data(dir_train)
representation_test =  treat_data(dir_test)

small_train = dict()
small_test = dict()

# 通过修改i, j 修改训练的类别数
i = 0
for key, value in representation_train.items():
    if i < 3:  # 修改3 修改类别数
        small_train[key] = value
        i += 1
    else:
        break
        
j = 0
for key, value in representation_test.items():
    if j < 3:
        small_test[key] = value
        j += 1
    else:
        break

print(small_test)

In [ ]:
def sigmoid(x, alpha):
    s_x = []
    for i in range(len(x)):
        if alpha * x[i] > -700:
            s_x.append(1.0 / (1 + np.exp(-alpha * x[i])))
        else:
            s_x.append(np.exp(alpha * x[i]) / (1 + np.exp(alpha * x[i]))) # for result the problem of overflow encountered in exp

    res_x = np.array(s_x)
    return res_x

In [ ]:
def softmax(x0,x):
    ex = []
    for i in range(len(x)):
        ex.append(np.exp(x[i]))
    return np.exp(x0)/sum(ex)

In [ ]:
class model_NN:

    def __init__(self, data_dict):
        self.data_dict = data_dict
        data_class_total_arr = []
        n = 0
        
        for item in self.data_dict:
            data_class_total_arr.append(self.data_dict[item])

        data_class_total_list = []

        for data_class_idem_arr in data_class_total_arr:
            data_class_idem_list = []
            for image0 in data_class_idem_arr:
                image0 = image0.tolist()  # 2-dimension list for each image
                image = []
                n = n + 1
                for line in image0:
                    for unit in line:
                        image.append(unit)  # 1-dimension list for each image
                # print(image)
                data_class_idem_list.append(np.array(image))
            data_class_total_list.append(data_class_idem_list)

        self.data_total_extend = data_class_total_list
        # invert all data to a total list which includes lists of classed-images, and the image is a 1-dimension array

        first_class = self.data_total_extend[0]
        # first class of images in total list

        self.image_dim = len(first_class[0])
        self.image_num = n


    def forward_propagation(self, sigmoid_alpha, label_index, image, weight_list):
        # num_layer=1  !!!Attention: layer_dim should be bigger than 1
        # val_error_total = 0

        # temps = self.data_total_extend[label_index]
        # image = temps[image_index]

        x0 = image
        print('x0',x0)
        #x0_act = sigmoid(x0, sigmoid_alpha)
        w0 = weight_list[0]
        print('w0',w0)
        #w0 = np.random.rand(layer_dim, self.image_dim)

        x1 = np.matmul(w0, x0)
        print('x1',x1)
        x1_act = sigmoid(x1, sigmoid_alpha)
        print('x1_act',x1_act)
        w1 = weight_list[1]
        #w1 = np.random.rand(len(self.data_total_extend), layer_dim)

        x_out = np.matmul(w1, x1_act)
        print('x_out',x_out)
        #x_out_act = sigmoid(x_out, sigmoid_alpha)
        x_out_act_list = []
        for m in range(len(x_out)):
            x_out_act_list.append(softmax(x_out[m],x_out))
        x_out_act = np.array(x_out_act_list)
        print('x_out_act', x_out_act)

        standard_list = []
        for i in range(len(x_out_act)):
            if i == label_index:
                standard_list.append(1)
            else:
                standard_list.append(0)
        standard = np.array(standard_list)

        val_error_list = []
        #val_error_abs_list =[]
        val_error = 0
        for i in range(len(x_out_act)):
            val_error_list.append(x_out_act[i] - standard[i])
            # val_error_abs_list.append(abs(val_error_list[i]))
            val_error += val_error_list[i] ** 2  # calculation of error value for each image
        # print(val_error)
        # val_error_total += val_error
        x_out_list = x_out.tolist()
        res_class = x_out_list.index(max(x_out_list)) # give the result of class

        x_vect_list = [x0, x1, x_out]
        x_act_vect_list = [x1_act, x_out_act]

        result = [x_vect_list, x_act_vect_list, standard, val_error,res_class]
        return result

    def backward_prograpation(self, x_vect_list, x_act_vect_list, standard, sigmoid_alpha, weight_list):

        x0 = x_vect_list[0]
        x1 = x_vect_list[1]
        x_out = x_vect_list[2]
        #x0_act = x_act_vect_list[0]
        x1_act = x_act_vect_list[0]
        x_out_act = x_act_vect_list[1]

        Dx_out_act = -2 * (standard - x_out_act)
        
        Dx_out_act_xout = []
        sum = 0
        for i in range(len(x_out)):
            sum += np.exp(x_out[i])
        for i in range(len(x_out)):
            A = sum - np.exp(x_out[i])
            Dx_out_act_xout.append(softmax(x_out[i],x_out)* A /(np.exp(x_out[i]) + A))
            
        Dx_out_act_to_x_out = np.array(Dx_out_act_xout)           
        # Dx_out_act_to_x_out = sigmoid_alpha * np.exp(-sigmoid_alpha * x_out) / ((1 + np.exp(-sigmoid_alpha * x_out)) ** 2)
        Dx_out = Dx_out_act * Dx_out_act_to_x_out
        Dw1 = np.multiply(Dx_out.reshape(-1, 1), x1_act)

        # Dx_out_to_x_1_act = w1
        Dx1_act_to_x1 = sigmoid_alpha * np.exp(-sigmoid_alpha * x1) / ((1 + np.exp(-sigmoid_alpha * x1)) ** 2)
        # Dw0 = np.multiply((Dx_out * Dx1_act_to_x1).reshape(-1, 1), x0_act)
        w1 = weight_list[1]
        # Dw0 = Dw0 * w1
        mid = (np.matmul(Dx_out, w1) * Dx1_act_to_x1)
        Dw0 = np.multiply(mid.reshape(-1, 1), x0)

        grad_list = [Dw0, Dw1]

        return grad_list

    def gradient_pas_fixe(self, pas, para_matrix_list, grad_matrix_list):
        para_new_list = []

        for i in range(len(para_matrix_list)):
            para_matrix = para_matrix_list[i]
            grad_matrix = grad_matrix_list[i]
            para_new = para_matrix - pas * grad_matrix
            para_new_list.append(para_new)

        return para_new_list


    def train(self, sigmoid_alpha, layer_dim, pas, precision):
        sig_a = sigmoid_alpha
        p = pas
        eps = precision * self.image_num
        iteration = 0
        #w_storage_image = []
        #w_storage_class = []
        error = 0

        while (error > eps) or (iteration == 0):
            error = 0
            for label_i in (range(len(self.data_total_extend))):
             #   if iteration != 0:
              #      weight_list_mid = w_storage_class[label_i]

                #w_storage_image = []

                for image_i in range(len(self.data_total_extend[label_i])):

                    image_mid = self.data_total_extend[label_i]
                    image = image_mid[image_i]

                    if iteration == 0 and label_i == 0 and image_i == 0:
                        weight_list = [-1 + 2 * np.random.rand(layer_dim, self.image_dim), -1 + 2 * np.random.rand(len(self.data_total_extend), layer_dim)]

                    forward_result = self.forward_propagation(sig_a, label_i, image, weight_list)
                    error_temp = forward_result[3]
                    grad_list = self.backward_prograpation(forward_result[0], forward_result[1], forward_result[2], sig_a, weight_list)
                    weight_list = self.gradient_pas_fixe(p, weight_list, grad_list)

                    error += error_temp
                    #w_storage_image.append(weight_list)

                #w_storage_class.append(w_storage_image)
            print("The iteration time is:",iteration)
            iteration += 1

        w_trained = weight_list
        return w_trained


    def predict(self, sigmoid_alpha, data_test_dict, w_trained):

        sig_a = sigmoid_alpha
        data_test_extend = []
        data_mid = []
        error_list = []
        result_class_list = []

        for item in data_test_dict:
            data_mid.append(data_test_dict[item])      # 把所有类的数据都放在同一个列表里，每个类的列表里都有多张图片，图片是以二维数组的形式存在

        for label_mid in data_mid:
            label = []
            for image2 in label_mid:
                image2 = image2.tolist()  # 2-dimension list for each image
                image1 = []
                for line in image2:
                    for unit in line:
                        image1.append(unit)  # 1-dimension list for each image
                # print(image)
                label.append(np.array(image1))
            data_test_extend.append(label)     # data_test_extend 将所有图片变成一维


        for label_i in range(len(data_test_extend)):

            error_list_mid = []
            result_class_list_mid = []

            for image_i in range(len(data_test_extend[label_i])):

                image_mid = data_test_extend[label_i]
                image = image_mid[image_i]
                #weight_list_mid = w_trained[label_i]
                weight_list = w_trained
                forward_result = self.forward_propagation(sig_a, label_i, image, weight_list)
                error = forward_result[3]
                result_class = forward_result[4]
                error_list_mid.append(error)
                result_class_list_mid.append(result_class)
                

            error_list.append(error_list_mid)
            result_class_list.append(result_class_list_mid)

        result = error_list
        # val_result = 0
        right_cnt = 0

        for l in range(len(error_list)):
            print ("For class",l,"'s imgages :\n")
            for i in range(len(error_list[l])):
                mid = result_class_list[l]
                mid1 = error_list[l]
                print ("    image",i,":     result of class is:",mid[i],"   error is:",mid1[i],'\n')
                
                if mid[i] == l:
                    right_cnt += 1
        
        val_result = right_cnt/self.image_num
        
        
        print ('\n\nThe result of precision is:',val_result)
        print ("\n-----------------------------------------Program FINISHED-------------------------------------------")

        return result

In [ ]:
Train = model_NN(small_train)
Test = model_NN(small_test)

In [ ]:
#################################CHANGE Parameters######################################################################
sigmoid_alpha = 1
layer_dim = 128
# Attention: layer dimension should be bigger than 1, because the error will be caused with layer_dim = 1
precision = 0.1 # change precision here!

In [ ]:
print("\n-------------------------------------------Program START--------------------------------------------")
print("With the coefficient of sigmoid function is:", sigmoid_alpha,'\n')


#weight_list = [np.random.rand(layer_dim, x.image_dim), np.random.rand(len(x.data_total_extend), layer_dim)

print("---------------------------------------------TRAINING----------------------------------------------\n\n")
w = Train.train(sigmoid_alpha, layer_dim, 0.2, precision) 
print("---------------------------------------The Result of train---------------------------------------\n")
print("Weight matrix are: ")
print(w)

print("\n---------------------------------------The Result of Predict---------------------------------------\n")
print("with precision of", precision)
Test.predict(sigmoid_alpha, small_test, w)